In [3]:
import pickle

import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Conv1D, BatchNormalization, ReLU, Dropout, Add, MaxPooling1D

from tensorflow.keras.layers import Bidirectional,LSTM

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape

from tensorflow.keras import Model
from tensorflow.keras import Input


le_file_path = 'evenet_data.pkl'

# Load data from the pickle file
with open(le_file_path, 'rb') as file:
    loaded_data = pickle.load(file)


new_data, classes = loaded_data

x = new_data[0]

def residual_block(x, filters, kernel_size, dropout_rate):
    # Convolutional layer
    conv = Conv1D(filters=filters,kernel_size= kernel_size,strides=2, 
                            activation=None,padding='same')(x)
    bn = BatchNormalization()(conv)
    relu = ReLU()(bn)
    dropout = Dropout(dropout_rate)(relu)

    fin = Conv1D(filters=filters,kernel_size= kernel_size, 
                            activation=None,padding='same')(dropout)

    x = MaxPooling1D(pool_size=2)(x)
    
    # Residual connection
    residual = Add()([x, fin])
    return residual


def residual_block2(x, filters, kernel_size, dropout_rate):
    
    #print(x.shape)
    X = BatchNormalization()(x)
    #print(X.shape)
    X = ReLU()(X)
    #print(X.shape)
    X = Dropout(dropout_rate)(X)
    #print(X.shape)
    X = Conv1D(filters=filters,kernel_size= kernel_size,strides=2, 
                            activation=None,padding='same')(X)
    #print(X.shape)
    X = BatchNormalization()(X)
    #print(X.shape)
    X = ReLU()(X)
    #print(X.shape)
    X = Dropout(dropout_rate)(X)
    #print(X.shape)
    X = Conv1D(filters=filters,kernel_size= kernel_size, 
                            activation=None,padding='same')(X)
    #print(X.shape)
    x = MaxPooling1D(pool_size=2)(x)
    #print(x.shape)

    # Residual connection
    residual = Add()([x, X])
    return residual


In [31]:
print(new_data[1], classes[1])
print(len(new_data), len(classes))

[-12. -15. -18. ... -63. -64. -64.] [1, 0, 0, 0]
122550 122550


In [4]:
inputs = tf.keras.Input(shape=(1024,1))
Conv = Conv1D(filters=64,kernel_size= 16,strides=2,activation=None,padding='same')(inputs)
bn = BatchNormalization()(Conv)
relu = ReLU()(bn)

#print(relu.shape)
#model = Model(inputs,relu)

res = residual_block(relu,64,16,0.2)

#print(res.shape)
#model = Model(inputs,res)

num_residual_layers = 4

for i in range(num_residual_layers):
    res = residual_block2(res,64,16,0.2)
    #print(res.shape)

#model = Model(inputs,res)

last_layer = BatchNormalization()(res)
last_layer = ReLU() (last_layer)
last_layer = Bidirectional(LSTM(64, return_sequences=True))(last_layer)
#print(last_layer.shape)

last_layer = Dropout(0.2)(last_layer)

features = Dense(1, activation='sigmoid')(last_layer)
features = (Reshape((16,), input_shape=(1, 16, 1)))(features)

#print(features.shape)
#model = Model(inputs,features)

logits = Dense(units=4, activation='softmax') (features)
#print(logits.shape)
model = Model(inputs,logits)


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

# works up to here


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1024, 1)]            0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 512, 64)              1088      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 512, 64)              256       ['conv1d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 512, 64)              0         ['batch_normalization[0][0

In [5]:
x_train = new_data[0]
x_train = tf.reshape(x_train, shape=(1,1024,1)) 
y_train = tf.constant([1,0,0,0],shape=(1,4))

x_val = new_data[1]
x_val = tf.reshape(x_val, shape=(1,1024,1)) 
y_val = tf.constant([1,0,0,0],shape=(1,4))



history = model.fit(x=x_train,y=y_train,epochs=10,batch_size=1,validation_data=(x_val,y_val))
history

Epoch 1/10
1/1 [==============================] - 13s 13s/step - loss: 1.7483 - accuracy: 0.0000e+00 - val_loss: 1.5838 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 80ms/step - loss: 1.4927 - accuracy: 0.0000e+00 - val_loss: 1.5097 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 93ms/step - loss: 1.3022 - accuracy: 0.0000e+00 - val_loss: 1.3842 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 87ms/step - loss: 1.1373 - accuracy: 0.0000e+00 - val_loss: 1.3742 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 97ms/step - loss: 1.0114 - accuracy: 1.0000 - val_loss: 1.3776 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 90ms/step - loss: 0.9230 - accuracy: 1.0000 - val_loss: 1.3728 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 104ms/step - loss: 0.8275 - accuracy: 1.0000 - val_loss: 1.3900 - val_acc

In [30]:
x_data = new_data[:1000]
y_data = classes[:1000]

##shuffle
import random
zipped = list(zip(x_data, y_data))
random.shuffle(zipped)
x_data, y_data = zip(*zipped)
#print(tf.shape(X_train), tf.shape(Y_train))
#print(len(X_train), len(Y_train))

X_train = x_data[:300]
Y_train = y_data[:300]
X_train = tf.convert_to_tensor(X_train)
X_train = tf.reshape(X_train, shape=(len(X_train),1024,1)) 
Y_train = tf.convert_to_tensor(Y_train)
Y_train = tf.reshape(Y_train, shape=(len(Y_train),4,1))


X_val = x_data[900:1000]
Y_val = y_data[900:1000]
X_val = tf.convert_to_tensor(X_val)
X_val = tf.reshape(X_val,shape=(len(X_val),1024,1))
Y_val = tf.convert_to_tensor(Y_val)
Y_val = tf.reshape(Y_val,shape=(len(Y_val),4,1))



history = model.fit(x=X_train,y=Y_train,epochs=10,batch_size=1,validation_data=(X_val,Y_val))
print(model.predict(new_data[400]))
print(classes[400])

Epoch 1/10
300/300 [==============================] - 9s 31ms/step - loss: 1.6248 - accuracy: 0.2633 - val_loss: 2.0365 - val_accuracy: 0.2200
Epoch 2/10
300/300 [==============================] - 9s 31ms/step - loss: 1.4871 - accuracy: 0.2633 - val_loss: 1.4718 - val_accuracy: 0.2500
Epoch 3/10
300/300 [==============================] - 9s 31ms/step - loss: 1.3143 - accuracy: 0.3167 - val_loss: 1.4270 - val_accuracy: 0.2300
Epoch 4/10
300/300 [==============================] - 9s 31ms/step - loss: 1.1732 - accuracy: 0.4167 - val_loss: 1.4242 - val_accuracy: 0.2800
Epoch 5/10
300/300 [==============================] - 9s 30ms/step - loss: 1.0038 - accuracy: 0.4933 - val_loss: 1.4535 - val_accuracy: 0.2900
Epoch 6/10
300/300 [==============================] - 9s 31ms/step - loss: 0.9438 - accuracy: 0.5133 - val_loss: 1.4247 - val_accuracy: 0.2500
Epoch 7/10
300/300 [==============================] - 10s 32ms/step - loss: 0.7899 - accuracy: 0.6933 - val_loss: 1.3636 - val_accuracy: 0.350

ValueError: in user code:

    File "C:\Users\EK\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\EK\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\EK\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\EK\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 2283, i

In [ ]:
#to save the model do this: 

model.save("myModel.h5")

In [ ]:
# to load a model do this:

loaded_model = tf.keras.models.load_model("myModel.h5")

loaded_model.summary()

In [ ]:
second_to_last_name = loaded_model.layers[-2].name

feature_extraction_model = Model(inputs = loaded_model.input,outputs=loaded_model.get_layer(second_to_last_name).output)

input_data = new_data[5]
input_data = tf.reshape(input_data, shape=(1,1024,1))

features = feature_extraction_model(input_data)

features